In [3]:
from utils.sampling import *
from utils.config import cfg_from_yaml_file
from tools import builder
from datasets.data_transforms import ResamplePoints, LidarSimulation
from datasets import build_dataset_from_cfg
import open3d as o3d
import numpy as np
import glob
from utils.vis_utils import convert_to_o3dpcd
from pathlib import Path
import pickle
from tqdm import tqdm

In [11]:


data_dir = Path('/VCN/data/shapenet/VC/vc-surface_nviews-20/val')
modelid_path = data_dir / 'model_ids.txt'

with open(modelid_path, 'r') as f:
    model_ids = [line.split('\n')[0] for line in f.readlines()]

partial_points_path = str(data_dir / 'partial' / '%s' / '%03d.pcd')
surface_points_path = str(data_dir / 'surface' / '%s' / '%03d.pcd')
label_path = str(data_dir / 'label.pkl')

with open(label_path, 'rb') as f:
    label = pickle.load(f)

In [14]:
from tqdm import tqdm 

lidarsim = LidarSimulation({})

dataset_size = 2000
data_dir = Path('/VCN/data/shapenet/VC/vc-surface_nviews-20/test')
partial_dir = Path('/VCN/data/shapenet/VC/vc-surface_nviews-20/test/partial')
surface_dir = Path('/VCN/data/shapenet/VC/vc-surface_nviews-20/test/surface')
num_exported = 0

new_labels = {}
for i in tqdm(range(len(model_ids)), total=len(model_ids)):
    
    m_id = model_ids[i]  
    new_labels[m_id] = {}
    
    for v_id in range(20):             
        partial_path = partial_points_path % (m_id, v_id)
        surface_path = surface_points_path % (m_id, v_id)

        pcd = o3d.io.read_point_cloud(partial_path)
        pc_np = np.asarray(pcd.points)
        pc_np_sim = lidarsim(pc_np)
        partial_pc_path = partial_dir / m_id
        partial_pc_path.mkdir(exist_ok=True, parents=True)        
        o3d.io.write_point_cloud(str(partial_pc_path / f'{v_id:03d}.pcd'), convert_to_o3dpcd(pc_np_sim))
        
        surface_pcd = o3d.io.read_point_cloud(surface_path)
        surface_pc_path = surface_dir / m_id
        surface_pc_path.mkdir(exist_ok=True, parents=True) 
        o3d.io.write_point_cloud(str(surface_pc_path / f'{v_id:03d}.pcd'), surface_pcd)
        
        new_labels[m_id][v_id] = label[m_id][v_id]
        new_labels[m_id][v_id]['num_pts'] = pc_np_sim.shape[0]        

with open(str(data_dir / 'label.pkl'), 'wb') as f:
    pickle.dump(new_labels, f)

100%|█████████████████████████████████████████████| 340/340 [01:22<00:00,  4.10it/s]


In [6]:
Path(label_dir).exists()

False

In [13]:
dataset = 'nuscenes'
label_dir = f'/VCN/data/VC/test_5000/{dataset}/test/label'
pc_dir = f'/VCN/data/VC/test_5000/{dataset}/test/partial'
label_pths = glob.glob(label_dir + '/*.pkl')

combined = {}
for pth in label_pths:
    with open(pth, 'rb') as f:
        label = pickle.load(f)
    
    pc_id = label['pc_id']
    pc = o3d.io.read_point_cloud(pc_dir + f'/{pc_id}.pcd')
            
    combined[pc_id] = label
    combined['dataset'] = dataset
    combined[pc_id]['num_pts'] = len(pc.points)    
    
new_lab_path = f'/VCN/data/VC/test_5000/{dataset}/test/label.pkl'
with open(new_lab_path, 'wb') as f:
    pickle.dump(combined, f)

In [14]:
with open(new_lab_path, 'rb') as f:
    plabel = pickle.load(f)

In [15]:
plabel

{'frame-125_car-004': {'bbox_pts': array([[14.36626367, 24.44785965, -0.17065461],
         [10.11437218, 23.12811618, -0.17065461],
         [14.9511365 , 22.56354206, -0.17065461],
         [14.36626367, 24.44785965,  1.59834539],
         [10.69924502, 21.2437986 ,  1.59834539],
         [14.9511365 , 22.56354206,  1.59834539],
         [10.11437218, 23.12811618,  1.59834539],
         [10.69924502, 21.2437986 , -0.17065461]]),
  'gtbox': array([12.53275434, 22.84582912,  0.51384539,  4.452     ,  1.973     ,
          1.769     , -2.84063146]),
  'pc_id': 'frame-125_car-004',
  'num_pts': 11},
 'dataset': 'nuscenes',
 'frame-134_car-011': {'bbox_pts': array([[ 7.66932026, -2.41267692, -1.56071684],
         [ 7.42630744,  2.13383314, -1.56071684],
         [ 6.01568076, -2.50106463, -1.56071684],
         [ 7.66932026, -2.41267692, -0.14171684],
         [ 5.77266793,  2.04544544, -0.14171684],
         [ 6.01568076, -2.50106463, -0.14171684],
         [ 7.42630744,  2.13383314, -0